In [1]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('research_traffic_multiple_labels.csv')
print(df.shape)
df.head()

(60000, 12)


,Count of Source IP,Port Count,Pair Count Ratio,Packet Count Diff,Lookup Count Diff,Protocol,Average Packet Count,Average Byte Count,Packet Std Dev,Byte Std Dev,Duration per Flow,Label
0,545,545,0.0,2,43,6,0.027523,4.788991,0.163601,28.466647,0.668020,3
1,545,545,0.0,10,0,6,0.009174,1.596330,0.095342,16.589551,0.419081,3
2,52,52,0.0,3,0,6,0.000000,0.000000,0.000000,0.000000,0.018792,3
3,545,545,0.0,13,21,6,0.023853,4.150459,0.355016,61.772825,2.946281,3
4,545,545,0.0,2,131,6,0.027523,4.788991,0.163601,28.466647,2.674461,3


In [2]:
import pickle
import numpy as np
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)
from datetime import datetime

y = df.Label
X = df.drop(['Label'],axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
print("Data normalized and split.")

Dataset normalized using MinMaxScaler.
Data normalized and split.


In [3]:
print(y_unseen_test.value_counts())

Label
1    2056
0    2009
2    2005
3    2004
4    1965
5    1961
Name: count, dtype: int64


In [4]:
from deap import base, creator, tools, algorithms
from sklearn.metrics import accuracy_score

def evaluate(weights):
    weighted_predictions = []
    for i in range(len(y_unseen_test)):
        class_votes = {}
        for j, (name, model) in enumerate(models):
            pred = predictions[name][i]
            class_votes[pred] = class_votes.get(pred, 0) + weights[j]
        weighted_predictions.append(max(class_votes, key=class_votes.get))
    acc = accuracy_score(y_unseen_test, weighted_predictions)
    return (acc,)


In [5]:
models = []
start = datetime.now()
pickle_files = ["xgb_multi.pkl", "lgb_multi.pkl", "rf_multi.pkl", "ann_multi.pkl", "knn_multi.pkl", "dt_multi.pkl", "nb_multi.pkl", "lr_multi.pkl", "svm_multi.pkl"]
for filename in pickle_files:
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    model_name = filename.replace("_multi.pkl", "")
    models.append((model_name, model))
print(f"{len(models)} models loaded successfully.")

9 models loaded successfully.


In [6]:
predictions = {}
for name, model in models:
    if name == "ann":
        prediction = np.array((model.predict(X_unseen_test) > 0.5).astype(int))
    else:
        prediction = np.array(model.predict(X_unseen_test))
    if prediction.ndim > 1 and prediction.shape[1] > 1:  # if not 1-dimensional, flatten it
        prediction = np.argmax(prediction, axis=1)
    elif prediction.ndim > 1:
        prediction = prediction.ravel()
    predictions[name] = prediction

    acc = accuracy_score(y_unseen_test, prediction)
    prec = precision_score(y_unseen_test, prediction, average='weighted')
    rec = recall_score(y_unseen_test, prediction, average='weighted')
    f1  = f1_score(y_unseen_test, prediction, average='weighted')
    cm  = confusion_matrix(y_unseen_test, prediction)

    # Print the results for the current model
    print(f"Results for model: {name}")
    print("Accuracy: ", acc)
    print("Precision:", prec)
    print("Recall:   ", rec)
    print("F1 Score: ", f1)
    print("Confusion Matrix:")
    print(cm)
    print("-" * 40)
# # Optional: Inspect one of the predictions
# print(predictions['ann'])
# Step 2: Combine predictions and apply majority vote ensemble using pandas mode
# Create a DataFrame where each column corresponds to a model's predictions
pred_df = pd.DataFrame(predictions)

print("Predictions obtained from all models.")

# For each sample (row), get the mode (most common value) across the classifiers.
# Since there are 9 classifiers (an odd number), ties should not occur.
ensemble_predictions = pred_df.mode(axis=1)[0].values

# Step 3: Evaluate the ensemble predictions using standard metrics
accuracy  = accuracy_score(y_unseen_test, ensemble_predictions)
precision = precision_score(y_unseen_test, ensemble_predictions, average='weighted')
recall    = recall_score(y_unseen_test, ensemble_predictions, average='weighted')
f1        = f1_score(y_unseen_test, ensemble_predictions, average='weighted')
cm        = confusion_matrix(y_unseen_test, ensemble_predictions)

# Print out the results
print("Majority Vote Ensemble Evaluation (Pandas Mode):")
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)
print("F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

Results for model: xgb
Accuracy:  0.9550833333333333
Precision: 0.9570352162582053
Recall:    0.9550833333333333
F1 Score:  0.9538837766484161
Confusion Matrix:
[[1569   82    1   88    2  267]
 [  12 2021    0    0    0   23]
 [   1    1 2003    0    0    0]
 [   5    0    0 1999    0    0]
 [   3    1    0    0 1961    0]
 [  40   13    0    0    0 1908]]
----------------------------------------
Results for model: lgb
Accuracy:  0.95475
Precision: 0.9565394471248935
Recall:    0.95475
F1 Score:  0.9535446748169473
Confusion Matrix:
[[1569   82    2   90    2  264]
 [  12 2021    0    0    0   23]
 [   2    0 2003    0    0    0]
 [   7    0    0 1997    0    0]
 [   5    0    0    0 1960    0]
 [  41   13    0    0    0 1907]]
----------------------------------------
Results for model: rf
Accuracy:  0.9436666666666667
Precision: 0.944487940151616
Recall:    0.9436666666666667
F1 Score:  0.9424029494507786
Confusion Matrix:
[[1529   86    0  102    2  290]
 [  21 1993    0    0    0  

In [7]:
# Set up the Genetic Algorithm
num_models = len(models)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define population and individual initialization
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.rand)  # Random weight initialization

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=num_models)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run Genetic Algorithm
pop = toolbox.population(n=50)  # Population size
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)

# Extract best weights
best_individual = np.array(tools.selBest(pop, k=1)[0])
optimized_weights = best_individual / np.sum(best_individual)  # Normalize weights

# Store classifier names with optimized weights in a dictionary
classifier_names = [name for name, _ in models]
classifier_weights = {clf_name: weight for clf_name, weight in zip(classifier_names, optimized_weights)}

print("Optimized Classifier Weights:", classifier_weights)


gen	nevals
0  	50    
1  	28    
2  	38    
3  	34    
4  	29    
5  	24    
6  	35    
7  	30    
8  	35    
9  	28    
10 	26    
11 	30    
12 	42    
13 	37    
14 	31    
15 	33    
16 	23    
17 	36    
18 	32    
19 	33    
20 	28    
21 	34    
22 	30    
23 	32    
24 	23    
25 	28    
26 	28    
27 	21    
28 	36    
29 	32    
30 	30    
31 	31    
32 	34    
33 	28    
34 	32    
35 	26    
36 	31    
37 	35    
38 	32    
39 	36    
40 	32    
41 	25    
42 	32    
43 	14    
44 	27    
45 	26    
46 	25    
47 	33    
48 	30    
49 	36    
50 	24    
Optimized Classifier Weights: {'xgb': 0.47082968863626756, 'lgb': 0.3037283814863955, 'rf': 0.06867431323887492, 'ann': -0.16007424893521638, 'knn': -0.10914527642860268, 'dt': 0.2715298327978744, 'nb': 0.06768461737260743, 'lr': 0.18724737269604633, 'svm': -0.10047468086424723}


In [8]:
# List to store the final weighted ensemble predictions
weighted_ensemble_predictions = []
n_samples = len(y_unseen_test)

# Loop over each sample in the test set
for i in range(n_samples):
    weighted_votes = {}

    for name, prediction in predictions.items():
        pred_class = prediction[i]
        weight = classifier_weights.get(name, 1.0)
        # Accumulate the vote for the predicted class
        weighted_votes[pred_class] = weighted_votes.get(pred_class, 0) + weight

    final_class = max(weighted_votes, key=weighted_votes.get)
    weighted_ensemble_predictions.append(final_class)

accuracy  = accuracy_score(y_unseen_test, weighted_ensemble_predictions)
precision = precision_score(y_unseen_test, weighted_ensemble_predictions, average='weighted')
recall    = recall_score(y_unseen_test, weighted_ensemble_predictions, average='weighted')
f1        = f1_score(y_unseen_test, weighted_ensemble_predictions, average='weighted')
cm        = confusion_matrix(y_unseen_test, weighted_ensemble_predictions)

end = datetime.now()

print("Weighted Ensemble Evaluation:")
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)
print("F1 Score: ", f1)
print("Confusion Matrix:\n", cm)
print("training time: ", end-start)

Weighted Ensemble Evaluation:
Accuracy:  0.9555833333333333
Precision: 0.9575773893194756
Recall:    0.9555833333333333
F1 Score:  0.9543744387128484
Confusion Matrix:
 [[1570   81    1   90    2  265]
 [  10 2024    0    0    0   22]
 [   1    1 2003    0    0    0]
 [   4    0    0 2000    0    0]
 [   3    1    0    0 1961    0]
 [  39   13    0    0    0 1909]]
training time:  0:02:36.848713


In [9]:
# df_pred = pd.read_csv('multiple_test.csv')

# y_pred = df_pred.Label
# X_pred = df_pred.drop(['Label'],axis=1)

# X_pred_normalized = normalizer.fit_transform(X_pred)
# X_pred = pd.DataFrame(X_pred_normalized, columns=X_pred.columns)
# print("Dataset normalized using MinMaxScaler.")

In [10]:
# predictions_test = {}
# for name, model in models:
#     if name == "ann":
#         prediction= np.array((model.predict(X_pred) > 0.5).astype(int))
#     else:
#         prediction = np.array(model.predict(X_pred))
#     if prediction.ndim > 1 and prediction.shape[1] > 1:  # if not 1-dimensional, flatten it
#         prediction = np.argmax(prediction, axis=1)
#     elif prediction.ndim > 1:
#         prediction = prediction.ravel()
#     predictions_test[name] = prediction

#     acc = accuracy_score(y_pred, prediction)
#     prec = precision_score(y_pred, prediction, average='weighted')
#     rec = recall_score(y_pred, prediction, average='weighted')
#     f1  = f1_score(y_pred, prediction, average='weighted')
#     cm  = confusion_matrix(y_pred, prediction)

#     # Print the results for the current model
#     print(f"Results for model: {name}")
#     print("Accuracy: ", acc)
#     print("Precision:", prec)
#     print("Recall:   ", rec)
#     print("F1 Score: ", f1)
#     print("Confusion Matrix:")
#     print(cm)
#     print("-" * 40)
# # Optional: Inspect one of the predictions
# # print(predictions['ann'])
# # Step 2: Combine predictions and apply majority vote ensemble using pandas mode
# # Create a DataFrame where each column corresponds to a model's predictions
# X_meta_test = pd.DataFrame(predictions_test)

# print("Predictions obtained from all models.")


In [11]:
# import xgboost as xgb
# from sklearn.svm import SVC


# X_meta = pd.DataFrame(predictions)  # Features: predictions from classifiers
# y_meta = y_unseen_test  # True labels

# # Step 5: Train xgb as a Meta-Classifier
# meta_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=6, eval_metric='mlogloss', use_label_encoder=False)
# meta_clf.fit(X_meta, y_meta)

# # Step 6: Use Meta-Classifier to Make Final Predictions
# meta_predictions = meta_clf.predict(X_meta_test)

# # Step 7: Evaluate the Meta-Classifier Performance
# meta_accuracy = accuracy_score(y_pred, meta_predictions)
# meta_precision = precision_score(y_pred, meta_predictions, average='weighted')
# meta_recall = recall_score(y_pred, meta_predictions, average='weighted')
# meta_f1 = f1_score(y_pred, meta_predictions, average='weighted')
# meta_cm = confusion_matrix(y_pred, meta_predictions)

# # Print Results
# print("Meta-Classifier (xgb) Performance:")
# print(f"Accuracy: {meta_accuracy:.4f}")
# print(f"Precision: {meta_precision:.4f}")
# print(f"Recall: {meta_recall:.4f}")
# print(f"F1 Score: {meta_f1:.4f}")
# print("Confusion Matrix:\n", meta_cm)